# 🛑 Desarrollo a realizar

**Requisitos**

* Tarea de optimización:
  1.  Utilizar la clase Individual explicada en clase con los métodos y atributos vistos en los ejercicios prácticos.
  2. Justificar [explícitamente](https://dle.rae.es/expl%C3%ADcitamente) las decisiones tomadas respecto a los parámetros del algoritmo:
     * Tamaño de la población.
     * Estrategia de selección de padres (elitismo, ruleta, torneos,  etc.)
     * Método de reproducción (recombinación en un punto, en dos, uniforme,etc.).
     * Método de mutación (bit-string, flip-bit, etc.) y ratio
     * Estrategia de supervivencia entre poblaciones (estable, generacional, elitismo, etc.)
     * Condiciones de parada.
  3. Por cada iteración imprime el número de generación y los atributos del individuo con mejor aptitud.
  4. La representación ha de tener suficientes soluciones diversas y complejas, de modo que el espacio de búsqueda sea lo suficientemente grande como para permitir que el algoritmo genético explore múltiples combinaciones.
  5. La función de aptitud se ha de basar en al menos 3 variables complejas.
  6. **<font color="red">No es posible utilizar librerías.</font>**


**Consideraciones deseables**
* Crea una alternativa para:
  * Tipo de mutación
  * Método de reproducción
  * El sistema ha de tener un grado de sofisticación elevado.


**🤖Requisitos extra**

* Genéticos: Programa el código de tal manera que se puedan seleccionar distintos parámetros en cuanto:
  * Tasa de supervivencia.
  * Tasa de mutación.
  * Selección de padres.
  * Selección de tipo de mutación.
  * Selección de método de reproducción.
  * Tamaño de la población.


# 2 Optimización

## a) Problema a resolver
Somos un taller mecánico que reparamos diferentes piezas de coches. Cada pieza de coche tiene un tiempo de reparación estimado, y cada pieza tiene asignado un grupo de reparación. Luego, desde la pieza de reparación a la siguiente pieza de reparación hay un tiempo de viaje, menos en la útima pieza que es cuando finaliza el turno.

In [1]:
# Tiempos de reparación por pieza (en minutos)
tiempos_reparacion = [30, 40, 50, 25, 20, 45, 15]  # Focos, Frenos, Motor, Llantas, Batería, Suspensión, Aceite


grupo = [
    [1.0, 1.3, 1.2, 1.1, 0.9, 1.3, 1.5],  # Grupo A: Focos, Frenos, Motor, Llantas, Batería, Suspensión, Aceite
    [1.1, 1.0, 0.9, 1.3, 1.2, 1.1, 1.0],  # Grupo B: Focos, Frenos, Motor, Llantas, Batería, Suspensión, Aceite
    [1.2, 1.1, 1.0, 1.0, 1.1, 1.0, 1.2],  # Grupo C: Focos, Frenos, Motor, Llantas, Batería, Suspensión, Aceite
    [1.3, 1.0, 1.2, 1.1, 1.0, 1.3, 1.1]   # Grupo D: Focos, Frenos, Motor, Llantas, Batería, Suspensión, Aceite
]


matriz_tiempos = [
    [0, 10, 15, 20, 12, 18, 8],   # Focos a n
    [10, 0, 25, 30, 14, 20, 10],  # Frenos a n
    [15, 25, 0, 35, 18, 22, 12],  # Motor a n
    [20, 30, 35, 0, 16, 25, 15],  # Llantas a n
    [12, 14, 18, 16, 0, 19, 10],  # Batería a n
    [18, 20, 22, 25, 19, 0, 12],  # Suspensión a n
    [8, 10, 12, 15, 10, 12, 0]    # Aceite a n
]

## b) Representación de estados + Función de aptitud
A la hora de medir la aptitud de los individuos hemos insertado la función de fitness dentro de la clase.

Con la función **set_aptitud() evaluamos la aptitud de cada individuo de la población**. De esta manera, cada vez que queramos evaluar la aptitud de un individuo creado, simplemente tendremos que poner:

```
individuo.set_aptitud()
```

In [2]:
class Individuo:
    def __init__(self, cromosoma=None, aptitud=0):
        self._cromosoma = cromosoma
        self._aptitud = aptitud

    # Getter y Setter para cromosoma
    def get_cromosoma(self):
        return self._cromosoma

    def set_cromosoma(self, cromosoma):
        self._cromosoma = cromosoma

    # Getter y Setter para aptitud
    def get_aptitud(self):
        return self._aptitud

    def set_aptitud(self):
        indices_piezas = self._cromosoma[:7]  # Los primeros 7 elementos son los índices de las piezas
        factores_grupo = self._cromosoma[7]  # Sublista con los factores del grupo

        aptitud = 0

        for i in range(len(indices_piezas)):
            aptitud += tiempos_reparacion[indices_piezas[i]] * grupo[factores_grupo[i]][indices_piezas[i]]

            # Sumar tiempos entre piezas
            if i < len(indices_piezas) - 1:
                aptitud += matriz_tiempos[indices_piezas[i]][indices_piezas[i + 1]]


        self._aptitud = aptitud

## c) Función de generar cromosoma y población

- Generar Cromosoma: Con esta función generamos un cromosoma de 7 valores, este último contiene otro Array dentro que representa los grupos que realizan las acciones del primero. Es decir, para la posicion 0 del Array la realiza la posición 0 del Array que contiene la posición 7.

- Generar la población: Con esta función generamos la población, con el tamaño que se indique.

In [3]:
import random

def generar_cromosoma():
    # generamos 7 valores únicos entre 0 y 6
    primeras_siete = random.sample(range(7), 7)

    # generamos una Sublista en la posición 7, con valores entre 0 y 3
    sublista = [random.randint(0, 3) for _ in range(7)]

    return primeras_siete + [sublista]

def generar_poblacion(tamano):
    # Iniciamos la población
    poblacion = []
    for _ in range(tamano):
        cromosoma = generar_cromosoma()
        individuo = Individuo(cromosoma)
        individuo.set_aptitud()
        poblacion.append(individuo)
    return poblacion

#### Prueba unitaria Generar población

In [4]:
# Crear población de 500 individuos
poblacion = generar_poblacion(500)

# Ejemplo de impresión de cromosomas
for i, individuo in enumerate(poblacion[:5]):  # Mostrar los primeros 5
    print(f"Individuo {i+1}: {individuo.get_cromosoma()} Aptitud: {individuo.get_aptitud()}")

Individuo 1: [5, 4, 1, 6, 0, 3, 2, [2, 3, 3, 2, 0, 2, 3]] Aptitud: 344.0
Individuo 2: [4, 6, 3, 1, 2, 0, 5, [0, 0, 2, 1, 1, 2, 0]] Aptitud: 358.0
Individuo 3: [6, 2, 1, 4, 0, 3, 5, [0, 1, 1, 2, 2, 3, 2]] Aptitud: 346.0
Individuo 4: [1, 6, 4, 5, 3, 2, 0, [1, 1, 3, 0, 2, 3, 1]] Aptitud: 365.5
Individuo 5: [2, 4, 6, 1, 3, 0, 5, [2, 3, 0, 2, 0, 1, 0]] Aptitud: 361.5


## e) Selección de padres
Para la selección de padres hemos empleado el elitismo.

In [5]:
def seleccionar_padres(poblacion, tasa_supervivencia):
    # Ordenamos la población según la aptitud de manera descendente
    poblacion_ordenada = sorted(poblacion, key=lambda x: x.get_aptitud(), reverse=False)

    # Calculamos la cantidad de individuos a seleccionar según la variable tasa_supervivencia
    num_padres = int(tasa_supervivencia * len(poblacion))

    # Seleccionar el top de la tasa de supervivencia de individuos como padres
    padres = poblacion_ordenada[:num_padres]

    return padres

## f) Recombinación
Hemos empleado la **recombinación en un punto**, lo que hacemos es intercambiar los grupos entre los padres.
Ejemplo:

| **Padre**       | **Cromosoma Original** | **Descendiente** | **Cromosoma Resultante** |
|------------------|------------------------|-------------------|---------------------------|
| **Padre 1**      | `[4, 2, 3, 0, 1, 6, 5, [3, 2, 1, 0, 2, 0, 2]]` | **Descendiente 1** | `[4, 2, 3, 0, 1, 6, 5, [2, 3, 0, 2, 3, 2, 1]]` |
| **Padre 2**      | `[3, 4, 5, 6, 1, 0, 2, [2, 3, 0, 2, 3, 2, 1]]` | **Descendiente 2** | `[3, 4, 5, 6, 1, 0, 2,[3, 2, 1, 0, 2, 0, 2]]` |

También, para obtener mayor variabilidad en los datos obtenemos al primer padre, el mejor padre, y al último de la lista, el peor de los mejores. Así  el orden de las piezas no cambia y en caso de que la secuencia de trabajo sea buena igual el grupo de trabajo es mejorable, y al revés.

In [6]:
def recombinacion_en_un_punto(padres):

  descendientes = []
  y = len(padres)

  if (y % 2) != 0:  # Compruebo si son pares
    padres.pop()  # Eliminamos el último elemento para poder hacer la recombinación

  while padres:
    # Elimino el primer padre
    padre1 = padres.pop(0)
    # Elimino el último padre
    padre2 = padres.pop()
    #print(f"El padre es:{padre1._cromosoma}")
    #print(f"El padre2 es:{padre2._cromosoma}")

    P1 = padre1._cromosoma[0:7]
    G1 = padre1._cromosoma[7]
    P2 = padre2._cromosoma[0:7]
    G2 = padre2._cromosoma[7]

    descendiente1 = P1 + [G2]
    descendiente2 = P2 + [G1]

    individuo1 = Individuo(descendiente1)
    individuo2 = Individuo(descendiente2)

    individuo1.set_aptitud()
    individuo2.set_aptitud()

    #print(descendiente1, individuo1.get_aptitud())
    #print(descendiente2, individuo2.get_aptitud())

    descendientes.append(individuo1)
    descendientes.append(individuo2)


  return descendientes

### g) Mutación bit string
Hemos empleado esta mutación, para tener mayor variabilidad en los datos y porque al no ser una codificación binaria el hecho de cambiar toda la cadena a priori, no nos lleva a una mejor solución.

Para ello vamos a mutar los valores de los grupos para mayor variabilidad en los datos, y para que no obtengamos muchos valores de un mismo grupo y haya variedad.

Para el empleo de la mutación, tenemos dos funciones:
- **mutacion_bit_string**: Con esta función lo que hacemos es devolver el descendiente mutado según la tasa de mutación que proporciona la función inferior. 
- **mutacion_hijos**:  Con esta función se recogen los descendientes recombinados y devolvemos los descendientes mutados. Para ello, recorremos los descendientes y llamamos a la función anterior para que mute el individuo correspondiente según la tasa de mutación introducida en la función principal. 

In [7]:
def mutacion_bit_string(individuo, tasa_de_mutacion):

    P1 = individuo._cromosoma[0:7]
    grupo = individuo._cromosoma[7]
    #print(f"El grupo es: {grupo}")
    for i in grupo:
        # genero un número random de 0 a 1 y si es menor que la tasa de mutación que mute el gen 
        if random.random() < tasa_de_mutacion:

            # Cambia el gen por otro valor dentro del rango permitido
            valor_mutado = random.randint(0, 3)
            # Actualizamos la posición i del grupo
            grupo[i] = valor_mutado

    #print("Nuevo Grupo", grupo)
    genotipo_mutado = P1 + [grupo]
    #print(genotipo_mutado)
    # Actualizo en nuevo cromosoma
    individuo.set_cromosoma(genotipo_mutado)
    individuo.set_aptitud()
            
    return individuo  # retorno el individuo mutado

In [8]:
def mutacion_hijos(descendientes, tasa_mutacion):
    
    descendientes_mutados = []
    # recorremos los descendientes
    for individuo in descendientes:
        #print(individuo.get_cromosoma())

        individuo_mutado = mutacion_bit_string(individuo, tasa_mutacion)
        #print(individuo_mutado.get_cromosoma())

        descendientes_mutados.append(individuo_mutado)
        #print(f"El individuo nuevo es {individuo.get_cromosoma()[7]} {individuo.get_aptitud()}")
	
    descendientes_mutados = sorted(descendientes_mutados, key=lambda x: x.get_aptitud(), reverse=False)
    
    return descendientes_mutados

#### Prueba unitaria de cómo usar el bit-string

In [9]:
genotipo1 = [4, 2, 3, 0, 1, 6, 5, [3, 2, 1, 0, 2, 0, 2]]
individuo1 = Individuo(genotipo1)
tasa_mutacion = 0.40
print(f"El individuo sin mutar es {individuo1._cromosoma}")

individuo_nuevo = mutacion_bit_string(individuo1, tasa_mutacion)

print(f"El individuo mutado es: {individuo_nuevo.get_cromosoma()}")


El individuo sin mutar es [4, 2, 3, 0, 1, 6, 5, [3, 2, 1, 0, 2, 0, 2]]
El individuo mutado es: [4, 2, 3, 0, 1, 6, 5, [3, 3, 0, 1, 2, 0, 2]]


## Justificación de decisiones:

- Tamaño de la población: Hemos empleado una población de 10000 individuos  porque de 500 a 5000, los valores de los mejores individuos no se aproximaban.

- Estrategía de selección de padres: **Elitismo**. Pensamos en emplear la selección de la ruleta pero como los números puestos de trabajo no se pueden repetir, para nuestra problemática lo que mejor se adaptaba era seleccionar los mejores.
    Hemos empleado diferentes tasas de supervivencia para las diferentes **reproducciones** y la que más rendimiento nos ha dado ha para la reproducción **generacional ha sido 0.2** y para la **elitista a 0.25**.  

- Método de reproducción: **Recombinación en un punto**. Mayor justificación arriba.

- Método de mutación: **Mutación bit string**, con una tasa de mutación de 0,4 porque como solo recombinamos los grupos, de esta manera conseguimos mayor variabilidad en los datos. Mayor justificación de mutación arriba.

- Estrategia de supervivencia entre poblaciones: Para la supervivencia entre poblaciones hemos empleado dos para ver cual es la que más nos favorecía.

    - Generacional: con esta estrategia se reemplaza la totalidad de la población.

    - Elitismo: con esta estrategia mantenemos el 25% de los individuos + los descendientes mutados.

- Condiciones de parada: Para el Generacional hemos empleado 10 generaciones porque son suficientes. Pero para el Elitismo hemos tenido que reducir las generaciones porque la población se va reduciendo más o menos un 75% por cada iteración del bucle.


## h1) Función - GENERACIONAL

In [16]:
def main_generacional(tamaño_poblacion, tasa_supervivencia, tasa_mutacion, generaciones):

	# Inicializo la población inicial
	poblacion = generar_poblacion(tamaño_poblacion)

	for generacion in range(generaciones):
		# Seleccion de padres
		padres_seleccionados = seleccionar_padres(poblacion, tasa_supervivencia)
		elegido_sin_evolucionar = padres_seleccionados[0]
		print(f"Generación {generacion + 1} | El individuo mejor sin mutar {elegido_sin_evolucionar.get_cromosoma()} tiene una aptitud de {elegido_sin_evolucionar.get_aptitud()} ")

		#  Recombinación
		descendientes = recombinacion_en_un_punto(padres_seleccionados)
		# Mutación
		descendientes = mutacion_hijos(descendientes, tasa_mutacion)

		elegido = descendientes[0]
		print(f"Generación {generacion + 1} | El individuo {elegido.get_cromosoma()} tiene una aptitud de {elegido.get_aptitud()} ")

		# Vuelvo a generar la población
		poblacion = generar_poblacion(tamaño_poblacion)


## h1) Caso de uso - Generacional

In [17]:
tamaño_poblacion = 10000
tasa_supervivencia = 0.2
tasa_mutacion = 0.4
generaciones = 10
main_generacional(tamaño_poblacion,tasa_supervivencia, tasa_mutacion, generaciones)

Generación 1 | El individuo mejor sin mutar [3, 4, 0, 1, 6, 2, 5, [3, 3, 0, 3, 3, 1, 2]] tiene una aptitud de 306.0 
Generación 1 | El individuo [3, 4, 0, 2, 6, 1, 5, [2, 3, 0, 1, 3, 3, 2]] tiene una aptitud de 306.5 
Generación 2 | El individuo mejor sin mutar [5, 6, 1, 0, 2, 4, 3, [1, 2, 3, 0, 1, 0, 3]] tiene una aptitud de 309.0 
Generación 2 | El individuo [2, 0, 1, 6, 3, 4, 5, [1, 0, 3, 1, 3, 2, 2]] tiene una aptitud de 309.5 
Generación 3 | El individuo mejor sin mutar [3, 4, 2, 5, 6, 0, 1, [2, 0, 1, 2, 3, 1, 1]] tiene una aptitud de 308.5 
Generación 3 | El individuo [3, 4, 1, 6, 0, 2, 5, [2, 2, 1, 1, 0, 1, 2]] tiene una aptitud de 307.0 
Generación 4 | El individuo mejor sin mutar [5, 1, 6, 0, 2, 4, 3, [2, 3, 2, 0, 1, 0, 2]] tiene una aptitud de 308.0 
Generación 4 | El individuo [3, 4, 1, 0, 2, 6, 5, [2, 0, 1, 2, 2, 1, 2]] tiene una aptitud de 308.0 
Generación 5 | El individuo mejor sin mutar [2, 6, 5, 0, 1, 4, 3, [1, 1, 2, 1, 3, 3, 2]] tiene una aptitud de 305.0 
Generación 

## h2) Función - ELITISMO

In [14]:
def main_elitismo(tamaño_poblacion, tasa_supervivencia, tasa_mutacion, generaciones):
    # Generar la población inicial
    poblacion = generar_poblacion(tamaño_poblacion)

    for generacion in range(generaciones):

        # Seleccion padres
        padres_seleccionados = seleccionar_padres(poblacion, tasa_supervivencia)
        
        elegido_sin_evolucionar = padres_seleccionados[0]
        print(f"Generación {generacion + 1} | El individuo mejor sin mutar {elegido_sin_evolucionar.get_cromosoma()} tiene una aptitud de {elegido_sin_evolucionar.get_aptitud()} ")

        # Guardo los mejores padres de la población
        elites = padres_seleccionados

        # Recombinación
        descendientes = recombinacion_en_un_punto(padres_seleccionados)
        # Mutación
        descendientes = mutacion_hijos(descendientes, tasa_mutacion)
        #print(len(descendientes))
        # Mostrar el mejor individuo de la generación actual
        elegido = descendientes[0]
        print(f"Generación {generacion + 1} | El individuo {elegido.get_cromosoma()} tiene una aptitud de {elegido.get_aptitud()} ")

        # genero la nueva población
        poblacion = elites + descendientes
        #print("Tamaño de población: ",len(poblacion))

## h2) Caso de uso - Elitista

In [15]:
tamaño_poblacion = 10000
tasa_supervivencia = 0.25
tasa_mutacion = 0.4
generaciones = 5

main_elitismo(tamaño_poblacion,tasa_supervivencia, tasa_mutacion, generaciones)

Generación 1 | El individuo mejor sin mutar [3, 6, 4, 0, 1, 5, 2, [2, 1, 0, 1, 3, 2, 1]] tiene una aptitud de 310.0 
Generación 1 | El individuo [3, 4, 1, 0, 2, 6, 5, [2, 0, 3, 0, 1, 3, 2]] tiene una aptitud de 298.5 
Generación 2 | El individuo mejor sin mutar [3, 4, 1, 0, 2, 6, 5, [2, 0, 3, 0, 1, 3, 2]] tiene una aptitud de 298.5 
Generación 2 | El individuo [3, 4, 1, 0, 6, 2, 5, [3, 0, 3, 0, 1, 1, 2]] tiene una aptitud de 302.5 
Generación 3 | El individuo mejor sin mutar [3, 4, 1, 0, 6, 2, 5, [3, 0, 3, 0, 1, 1, 2]] tiene una aptitud de 302.5 
Generación 3 | El individuo [3, 4, 1, 0, 2, 6, 5, [1, 0, 3, 1, 1, 1, 2]] tiene una aptitud de 307.5 
Generación 4 | El individuo mejor sin mutar [3, 4, 1, 0, 2, 6, 5, [1, 0, 3, 1, 1, 1, 2]] tiene una aptitud de 307.5 
Generación 4 | El individuo [3, 4, 1, 0, 2, 6, 5, [2, 0, 1, 1, 1, 1, 2]] tiene una aptitud de 300.0 
Generación 5 | El individuo mejor sin mutar [3, 4, 1, 0, 2, 6, 5, [2, 0, 1, 1, 1, 1, 2]] tiene una aptitud de 300.0 
Generación 

## Algoritmo vs Fuerza Bruta
La primera celda muestra el código para resolver nuestra problematica por fuerza bruta. Tarda 10 minutos en ejecutarse.

In [ ]:
from itertools import permutations, product

# Datos
piezas = [0, 1, 2, 3, 4, 5, 6]  # Índices de las piezas
factores = [0, 1, 2, 3]         # Índices de los grupos

# Todas las permutaciones de las piezas
permutaciones_piezas = permutations(piezas)

# Todas las combinaciones posibles de factores del grupo (uno por cada pieza)
combinaciones_factores = list(product(factores, repeat=len(piezas)))

# Variable para rastrear el mejor tiempo
mejor_tiempo = float('inf')
mejor_configuracion = None

# Iterar sobre cada combinación de piezas y factores
for permutacion in permutaciones_piezas:
    for combinacion in combinaciones_factores:
        # Crear un cromosoma simulado
        cromosoma = list(permutacion) + [list(combinacion)]
        individuo = Individuo(cromosoma)
        individuo.set_aptitud()
        # Calcular la aptitud
        aptitud = individuo._aptitud  # Implementación basada en tu función set_aptitud
        
        # Verificar si es la mejor solución
        if aptitud < mejor_tiempo:
            mejor_tiempo = aptitud
            mejor_configuracion = cromosoma

# Resultado final
print(f"Mejor tiempo: {mejor_tiempo}")
print(f"Configuración óptima: {mejor_configuracion}")


Mejor tiempo: 297.0
Configuración óptima: [3, 4, 1, 0, 2, 6, 5, [2, 0, 1, 0, 1, 1, 2]]


Sin embargo, empleando el algoritmo creado tarda 2,... segundos de ejecución. Con la reproducción elitista llegamos a la mejor solución, por lo que para este caso nos favorece más y además, emplea menos tiempo que el generacional.
Los valores obtenidos son verídicos. Quizás alguno se pierde por ejecutar las celdas accidentalmente.

## TABLA COMPARATIVA

| **Método**       | **Cromosoma** | **Aptitud** |**Tiempo de ejecución** |
|-----------------|------------------------|------------|---|
| Reproducción **Generacional**       | *[3, 4, 1, 0, 2, 6, 5, [3, 0, 3, 0, 1, 1, 2]]* | **299.5**| 3s |
| Reproducción **Elistista**       | *[3, 4, 1, 0, 2, 6, 5, [2, 0, 1, 0, 1, 1, 2]]* | **297.0**| 0.2s |
| **FUERZA BRUTA**       | *[3, 4, 1, 0, 2, 6, 5, [2, 0, 1, 0, 1, 1, 2]]* | **297.0**| 1ª vez :10 m 2ª: 4m 16s|

